In [10]:
#import libraries and packages
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import time

In [11]:
!which chromedriver

/usr/local/bin/chromedriver


In [12]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# Mars News Title & Paragraph

In [13]:
#visiting the page
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [14]:
html = browser.html
soup = bs(html,"html.parser")

In [16]:
#Collect the News title + paragraph text
title = soup.find("div",class_="content_title").text
paragraph = soup.find("div", class_="article_teaser_body").text
print(f"Title: {title}")
print(f"Paragraph: {paragraph}")

Title: Global Storms on Mars Launch Dust Towers Into the Sky
Paragraph: A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze.


# Mars Image

In [26]:
url_nasa_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_nasa_image)

In [28]:
full_image = browser.find_by_id('full_image')
full_image.click()

In [29]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info = browser.find_link_by_partial_text('more info')
more_info.click()

In [31]:
html_code = browser.html
img_soup = bs(html_code, 'html.parser')

In [32]:
img_url = img_soup.select_one('figure.lede a img').get("src")
img_url

'/spaceimages/images/largesize/PIA01384_hires.jpg'

In [33]:
img_url_nasa = f'https://www.jpl.nasa.gov{img_url}'
img_url_nasa

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01384_hires.jpg'

In [27]:
# Scrape the browser into soup and use soup to find the image of mars
# Save the image url to a variable called `img_url`
html = browser.html
soup = bs(html, 'html.parser')

image = soup.find("img", class_="thumb")["src"]
img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(img_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.jpg')

In [18]:
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_image))
print(base_url)

https://www.jpl.nasa.gov/


In [19]:
#Design an xpath selector to grab the image from the web page
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"

In [20]:
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [21]:
#USe BeautifulSoup and get the image
html_image = browser.html
soup = bs(html_image, "html.parser")
img_url = soup.find("img", class_="fancybox-image")["src"]
full_img_url = base_url + img_url
print(full_img_url)

https://www.jpl.nasa.gov//spaceimages/images/largesize/PIA23561_hires.jpg


In [34]:
#get mars weather's latest tweet from the website
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [38]:
html_weather = browser.html
soup = bs(html_weather, "html.parser")
#Search for mars weather data
temp = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
print(mars_weather)
#temp

InSight sol 354 (2019-11-24) low -99.7ºC (-147.5ºF) high -22.6ºC (-8.6ºF)
winds from the SSW at 4.8 m/s (10.8 mph) gusting to 23.9 m/s (53.5 mph)
pressure at 6.80 hPapic.twitter.com/QojEKyxlhp


In [36]:
url_facts = "https://space-facts.com/mars/"

#  Mars Table

In [37]:
table = pd.read_html(url_facts)
table[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [39]:
df_mars_facts = table[0]
df_mars_facts.columns = ["Parameter", "Values"]
df_mars_facts.set_index(["Parameter"])

,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [40]:
mars_html_table = df_mars_facts.to_html()
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Parameter</th>      <th>Values</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronome

# Mars Hemispheres

In [60]:
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)

In [61]:
#Getting the base url
hemisphere_base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_hemisphere))
print(hemisphere_base_url)

https://astrogeology.usgs.gov/


In [62]:
hemi_image_urls = []
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hem = {}
    #Search elements
    browser.find_by_css("a.product-item h3")[i].click()
    # find sample image
    sample_element = browser.find_link_by_text('Sample').first
    hem['img_url'] = sample_element['href']
    
    # Get Hemisphere title
    hem['title'] = browser.find_by_css("h2.title").text
    # Append hemisphere object to list
    hemi_image_urls.append(hem)
    # Finally, we navigate backwards
    browser.back()

In [63]:
hemi_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]